# Memory

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [11]:
from langchain_core.chat_history import BaseChatMessageHistory

class InMemoryHistory(BaseChatMessageHistory):
    def __init__(self):
        self.messages = []   # 대화 저장하는 리스트

    def add_messages(self, messages):
        self.messages.extend(messages)
        
    def clear(self):
        self.messages = []
        
    def __repr__(self):
        return str(self.messages)
        

In [12]:
store = {}  # item(key=session_id, values=InMemoryHistory_인스턴스)

def get_by_session_id(session_id):
    if session_id not in store:
        store[session_id] = InMemoryHistory()
    return store[session_id]

In [13]:
history_test = get_by_session_id('test')
history_test.add_messages(['hello','good morning', 'how are you?'])
history_test.add_messages(['I am fine', 'Thank you'])

history_test

['hello', 'good morning', 'how are you?', 'I am fine', 'Thank you']

In [14]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory

prompt = ChatPromptTemplate.from_messages([
    ('system','너는 {skill}을 잘하는 AI 어시스턴트야.'),
    MessagesPlaceholder(variable_name='history'), # history라는 변수 이름에 이전 대화 내용 저장
    ('human','{query}')
])

model = ChatOpenAI(model_name='gpt-4o-mini', temperature=0.5)
chain = prompt | model 


In [16]:
# 이전 대화 내용을 저장하는 chain을 생성
chain_with_history = RunnableWithMessageHistory(
    chain,
    get_session_history = get_by_session_id,
    input_messages_key='query',
    history_messages_key='history'
)

In [17]:
response = chain_with_history.invoke(
    {'skill':'대화','query':'다람쥐는 도토리 나무를 세 그루 키우고 있습니다.'},
    config = {'configurable': {'session_id': 'squirrel'}}
)

print(response)

content='다람쥐가 도토리 나무를 세 그루 키우고 있다니, 정말 흥미로운 이야기네요! 도토리 나무는 다람쥐에게 중요한 식량 공급원이죠. 그들은 도토리를 저장해 두고 겨울철에 먹기도 합니다. 다람쥐가 나무를 잘 키우고 있다면, 건강한 도토리를 많이 얻을 수 있을 것 같네요. 혹시 다람쥐의 이야기에 대해 더 알고 싶으신가요?' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 115, 'prompt_tokens': 46, 'total_tokens': 161, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_06737a9306', 'finish_reason': 'stop', 'logprobs': None} id='run-55f84f70-747e-4229-99fa-db2acea6b67c-0' usage_metadata={'input_tokens': 46, 'output_tokens': 115, 'total_tokens': 161, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


In [18]:
store

{'test': ['hello', 'good morning', 'how are you?', 'I am fine', 'Thank you'],
 'squirrel': [HumanMessage(content='다람쥐는 도토리 나무를 세 그루 키우고 있습니다.', additional_kwargs={}, response_metadata={}), AIMessage(content='다람쥐가 도토리 나무를 세 그루 키우고 있다니, 정말 흥미로운 이야기네요! 도토리 나무는 다람쥐에게 중요한 식량 공급원이죠. 그들은 도토리를 저장해 두고 겨울철에 먹기도 합니다. 다람쥐가 나무를 잘 키우고 있다면, 건강한 도토리를 많이 얻을 수 있을 것 같네요. 혹시 다람쥐의 이야기에 대해 더 알고 싶으신가요?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 115, 'prompt_tokens': 46, 'total_tokens': 161, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_06737a9306', 'finish_reason': 'stop', 'logprobs': None}, id='run-55f84f70-747e-4229-99fa-db2acea6b67c-0', usage_metadata={'input_tokens': 46, 'output_tokens': 115, 'total_tokens': 161, 'input_token_detai

In [19]:
response = chain_with_history.invoke(
    {'skill':'대화','query':'토끼는 사과 나무를 다섯 그루 키우고 있습니다.'},
    config = {'configurable': {'session_id': 'squirrel'}}
)

print(response)

content='토끼가 사과 나무를 다섯 그루 키우고 있다니, 정말 귀엽고 흥미로운 설정이네요! 사과 나무는 토끼에게 맛있는 간식이 될 수 있겠어요. 사과는 토끼가 먹기에 좋은 과일 중 하나이니, 잘 자라서 풍성한 수확을 할 수 있기를 바랍니다. 혹시 이 이야기에 대해 더 궁금한 점이나 다른 동물들의 이야기도 나누고 싶으신가요?' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 111, 'prompt_tokens': 186, 'total_tokens': 297, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_06737a9306', 'finish_reason': 'stop', 'logprobs': None} id='run-38794574-4835-497f-827f-fbae7c49b4a5-0' usage_metadata={'input_tokens': 186, 'output_tokens': 111, 'total_tokens': 297, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


In [20]:
store

{'test': ['hello', 'good morning', 'how are you?', 'I am fine', 'Thank you'],
 'squirrel': [HumanMessage(content='다람쥐는 도토리 나무를 세 그루 키우고 있습니다.', additional_kwargs={}, response_metadata={}), AIMessage(content='다람쥐가 도토리 나무를 세 그루 키우고 있다니, 정말 흥미로운 이야기네요! 도토리 나무는 다람쥐에게 중요한 식량 공급원이죠. 그들은 도토리를 저장해 두고 겨울철에 먹기도 합니다. 다람쥐가 나무를 잘 키우고 있다면, 건강한 도토리를 많이 얻을 수 있을 것 같네요. 혹시 다람쥐의 이야기에 대해 더 알고 싶으신가요?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 115, 'prompt_tokens': 46, 'total_tokens': 161, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_06737a9306', 'finish_reason': 'stop', 'logprobs': None}, id='run-55f84f70-747e-4229-99fa-db2acea6b67c-0', usage_metadata={'input_tokens': 46, 'output_tokens': 115, 'total_tokens': 161, 'input_token_detai

In [21]:
response = chain_with_history.invoke(
    {'skill':'대화','query':'다람쥐와 토끼는 합해서 몇 그루의 나무를 키우고 있나요?'},
    config = {'configurable': {'session_id': 'squirrel'}}
)

print(response)

content='다람쥐가 키우는 도토리 나무가 3그루, 토끼가 키우는 사과 나무가 5그루이므로, 두 동물이 합쳐서 키우는 나무의 총 수는 3 + 5 = 8그루입니다. 다람쥐와 토끼가 함께 키우는 나무는 총 8그루네요!' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 89, 'prompt_tokens': 328, 'total_tokens': 417, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_06737a9306', 'finish_reason': 'stop', 'logprobs': None} id='run-b5a08471-e4d8-4732-9b31-11bd33d3b8f3-0' usage_metadata={'input_tokens': 328, 'output_tokens': 89, 'total_tokens': 417, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


# 단계별 Chatbot
- 첫 대화에서 내 이름을 알려주고, 다음 대화에서 내 이름 기억하는지 물어보기!
1. 그냥 Chat
    - langchain_openai의 ChatOpenAI
    - hain_core.messages의 클래스

In [22]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage('너는 대화를 잘하는 AI야.'),
    HumanMessage('안녕 내 이름은 이재혁이야.'),
]

model = ChatOpenAI(model_name='gpt-4o-mini', temperature=0.5)
response = model.invoke(messages)
print(response)

content='안녕하세요, 이재혁님! 만나서 반갑습니다. 어떻게 도와드릴까요?' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 31, 'total_tokens': 53, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_06737a9306', 'finish_reason': 'stop', 'logprobs': None} id='run-d881e32a-60fa-4a04-8ac2-9334ea82879a-0' usage_metadata={'input_tokens': 31, 'output_tokens': 22, 'total_tokens': 53, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


2. 직접 대화 맥락 유지
    - langchain_openai의 ChatOpenAI
    - langchain_core.messages의 클래스

In [33]:
from langchain_core.messages import AIMessage
message_history = [
    SystemMessage('너는 상황에 맞게 대답을 해주는 AI이고 같은 대답은 하지 않아.')
]
message_history.append(HumanMessage('안녕 내 이름은 이재혁이야.'))

model = ChatOpenAI(model_name='gpt-4o-mini', temperature=0.5)
response = model.invoke(messages)
print(response.content)


안녕하세요, 이재혁님! 만나서 반가워요. 어떻게 도와드릴까요?


In [34]:
message_history.append(AIMessage(response.content))
message_history.append(HumanMessage('내 이름 기억해?'))

response = model.invoke(message_history)
print(response.content)

print('======')
message_history.append(AIMessage(response.content))
print(message_history)

네, 이재혁님! 이름을 기억하고 있어요. 더 이야기하고 싶은 것이 있나요?
[SystemMessage(content='너는 상황에 맞게 대답을 해주는 AI이고 같은 대답은 하지 않아.', additional_kwargs={}, response_metadata={}), HumanMessage(content='안녕 내 이름은 이재혁이야.', additional_kwargs={}, response_metadata={}), AIMessage(content='안녕하세요, 이재혁님! 만나서 반가워요. 어떻게 도와드릴까요?', additional_kwargs={}, response_metadata={}), HumanMessage(content='내 이름 기억해?', additional_kwargs={}, response_metadata={}), AIMessage(content='네, 이재혁님! 이름을 기억하고 있어요. 더 이야기하고 싶은 것이 있나요?', additional_kwargs={}, response_metadata={})]


3. Memory로 대화 맥락 유지
- langchain_openai의 ChatOpenAI
- langchain_core.messages의 클래스
- langchain_core.runnables의 클래스
- langchain_core.prompts의 클래스

In [43]:
class InMemoryHistory():
    def __init__(self):
        self.messages = []   # 대화 저장하는 리스트

    def add_messages(self, messages):
        self.messages.extend(messages)
        
    def clear(self):
        self.messages = []
        
    def __repr__(self):
        return str(self.messages)

In [44]:
from langchain_core.runnables import RunnableWithMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

store = {}  # item(key=session_id, values=InMemoryHistory_인스턴스)

def get_by_session_id(session_id):
    if session_id not in store:
        store[session_id] = InMemoryHistory()
    return store[session_id]

In [45]:
prompt = ChatPromptTemplate.from_messages([
    ('system', '너는 상황에 맞게 대답을 해주는 AI이고 같은 대답은 하지 않아.'),
    MessagesPlaceholder(variable_name='history'),
    ('human','{query}')
])

model = ChatOpenAI(model_name='gpt-4o-mini', temperature=0.5)

chain = prompt | model

In [46]:
chain_with_history = RunnableWithMessageHistory(
    chain,
    get_session_history=get_by_session_id,
    input_messages_key='query',
    history_messages_key='history'
)

In [47]:
response = chain_with_history.invoke(
    {'query':'안녕 내 이름은 이재혁이야.'},
    config = {'configurable': {'session_id': 'lee'}}
)

print(response)

content='안녕하세요, 이재혁님! 만나서 반갑습니다. 어떤 이야기를 나눠보고 싶으신가요?' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 28, 'prompt_tokens': 42, 'total_tokens': 70, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_06737a9306', 'finish_reason': 'stop', 'logprobs': None} id='run-ed681211-2c89-45c6-86fd-a81ea531ba16-0' usage_metadata={'input_tokens': 42, 'output_tokens': 28, 'total_tokens': 70, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


In [48]:
store

{'lee': [HumanMessage(content='안녕 내 이름은 이재혁이야.', additional_kwargs={}, response_metadata={}), AIMessage(content='안녕하세요, 이재혁님! 만나서 반갑습니다. 어떤 이야기를 나눠보고 싶으신가요?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 28, 'prompt_tokens': 42, 'total_tokens': 70, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_06737a9306', 'finish_reason': 'stop', 'logprobs': None}, id='run-ed681211-2c89-45c6-86fd-a81ea531ba16-0', usage_metadata={'input_tokens': 42, 'output_tokens': 28, 'total_tokens': 70, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})]}

In [49]:
response = chain_with_history.invoke(
    {'query':'내 이름 기억해'},
    config = {'configurable': {'session_id': 'lee'}}
)

print(response)

content='이재혁님, 기억하겠습니다! 앞으로도 대화할 때 잊지 않을게요. 다른 궁금한 점이나 이야기하고 싶은 것이 있나요?' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 36, 'prompt_tokens': 81, 'total_tokens': 117, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_06737a9306', 'finish_reason': 'stop', 'logprobs': None} id='run-733e5653-24ec-4abd-a7fc-51a81cefa703-0' usage_metadata={'input_tokens': 81, 'output_tokens': 36, 'total_tokens': 117, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}
